# mdx仮想マシンへのsapporo-service構築
このNotebookでは、mdx仮想マシンのデプロイと、その仮想マシン上にsapporo-serviceを構築します。

## 準備

1. mdx REST API 認証トークン設定
2. mdx VM にSSHログインするためのキーペア作成
3. mdx の REST API エンドポイントに接続できることの確認

### mdx REST API 認証トークン設定

以下のセルを実行してmdx REST API認証トークンを入力します。

In [ ]:
from getpass import getpass
mdx_token = getpass('mdx API token')

### mdx VM への SSH ログイン用キーペア作成

mdx VM デプロイ時の設定項目に含まれる公開鍵を用意します。

In [ ]:
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""
!ls -l ~/.ssh

### mdx REST API エンドポイント接続確認

mdx REST APIエンドポイントにIPv6で接続しようとすると到達不可となる場合があるため、以下のセルを実行してIPv4での接続を強制します。

In [ ]:
# デフォルトのresolverがIPv6のアドレスを返すことにより以降のAPIで接続不可の場合があるため、以下のコードを実行しておく。
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

以下のセルを実行し、HTTPステータスコード 200 が返ることを確認します。

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

## mdx VM 作成

1. VCP SDK mdx用プラグインモジュールの読み込み
2. mdx VM作成に必要なパラメータ設定
3. mdx VMデプロイ実行

### VCP SDK mdx用プラグインモジュールの読み込み

In [ ]:
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### mdx VM作成に必要なパラメータ設定

- プロジェクトID
- ネットワークセグメントID
- sshログインのための公開鍵

自身が所属している（利用可能な）mdxのプロジェクト情報を確認します。

In [ ]:
import json
projects = mdx.get_assigned_projects()
for org in projects:
    print(json.dumps(org['projects'], indent=2, ensure_ascii=False))

以降のmdx操作対象とする「プロジェクト名」 (`name`) を設定します。

In [ ]:
# プロジェクト名
project_name = ''
mdx.set_current_project_by_name(project_name)

操作対象として設定したプロジェクト情報を確認します。

In [ ]:
print(json.dumps(mdx.get_current_project(), indent=2, ensure_ascii=False))

プロジェクトで利用可能なネットワークセグメントのリストを取得します。

In [ ]:
segments = mdx.get_segments()
print(json.dumps(segments, indent=2, ensure_ascii=False))

セグメントIDを取得します。

In [ ]:
# セグメント名
segment_name = ''

for segment in segments:
    if segment['name'] == segment_name:
        segment_id = segment['uuid']

segment_id

sshログインのための公開鍵ファイルの内容を設定します。

In [ ]:
import os
with open(os.path.expanduser('~/.ssh/id_rsa.pub')) as f:
    ssh_shared_key = f.read()
print(ssh_shared_key)

### mdx VMデプロイ

ここでは、仮想マシンテンプレートとして「推奨版、東京大学制作、20220412-2043版」を使用します。
> 利用可能な仮想マシンテンプレートの一覧は、 `get_vm_catalogs()` により取得可能

ここでは、一例として、[ハードウェアのカスタマイズ項目](https://docs.mdx.jp/ja/main/create_vm.html#deploy-settings)を以下の内容で設定します。  
各項目の意味は前述のリンク先（ハードウェアのカスタマイズ項目）の説明を確認してください。
  * 仮想マシン名: `"vcp-host0001"` **（プロジェクト内で重複しないこと）**
  * パックタイプ: `"cpu"`
  * パック数: `3`
  * 仮想ディスク(GB): `40`
  * ストレージネットワーク: `"portgroup"`
  * サービスレベル: `"spot"`

mdx VMにIPアドレスが設定されるまで5分程度要するため、実行中のセルの経過時間を表示するライブラリ [jupyter-autotime](https://pypi.org/project/jupyter-autotime/) 機能を有効化します。

In [ ]:
!pip install jupyter-autotime
%load_ext autotime

VMのパラメータを設定します。

In [ ]:
DEFAULT_CATALOG = "16a41081-a1cf-428e-90d0-a147b3aa6fc2"
DEFAULT_TEMPLATE_NAME = "UT-20220412-2043-ubuntu-2004-server"

vm_name = "sapporo"

mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=3,
    pack_type="cpu",
    disk_size=40,
    gpu="0",
    network_adapters=[
        dict(
            adapter_number=1,
            segment=segment_id
        )
    ],
    shared_key=ssh_shared_key,
    storage_network="portgroup",
    service_level="spot",
)

VMのデプロイを実行します。デプロイ完了後、VM情報を出力します。

In [ ]:
info = mdx.deploy_vm(vm_name, mdx_spec)
print(json.dumps(info, indent=2, ensure_ascii=False))

In [ ]:
# jupyter-autotime を無効化
%unload_ext autotime

VMに付与されたプライベートIPv4アドレスを取得します。

In [ ]:
private_ip_address = info["service_networks"][0]["ipv4_address"][0]
private_ip_address

プロジェクトに割り当てられたグローバルIPv4アドレスへの通信を仮想マシンについたプライベートIPv4アドレスに転送し、  
プロジェクト外部(インターネットなど)と仮想マシンが直接通信できるようにします。

使用可能なグローバルIPv4アドレス一覧を取得します。

In [ ]:
import requests

url = f'https://oprpl.mdx.jp/api/global_ip/project/{mdx._project_id}/assignable/'
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'JWT {mdx_token}',
}
res = requests.get(url, data=[], headers=headers)
res.json()

使用可能なグローバルIPv4アドレスを選びます。

In [ ]:
public_ip_address = ''

In [ ]:
import requests

url = 'https://oprpl.mdx.jp/api/dnat/'
dnat_spec = {
    'pool_address': public_ip_address,
    'segment': segment_id,
    'dst_address': private_ip_address,
}
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'JWT {mdx_token}',
}

res = requests.post(url, data=json.dumps(dnat_spec), headers=headers)
res

## VCP既存サーバ (SSH) モード セットアップ

1. mdx VMへの疎通確認
2. VM初期パスワード設定
3. 既存サーバ(SSH) モード セットアップスクリプトをmdx VMへのコピー
4. 既存サーバ(SSH) モード セットアップ実行
5. Docker のインストール確認

### mdx VMへの疎通確認

In [ ]:
!ping -c 3 {public_ip_address}

### VM初期パスワード設定

* 仮想マシンテンプレート「推奨版、東京大学制作、20220412-2043版」では、初回ログイン時にはOSのパスワード設定が求められる
* ログインユーザ名は `mdxuser`

VMの初期パスワード（ここでは例として `mdx_vm_initial_password` を指定）をSSH経由で設定するために必要なパラメータを設定します。

In [ ]:
ssh_user_name = "mdxuser"
ssh_private_key_path = os.path.expanduser("~/.ssh/id_rsa")
mdx_user_password = "mdx_vm_initial_password"

expectスクリプトを用いてVMの初期パスワード設定を実行します。

In [ ]:
import pathlib
dir = pathlib.Path(os.getcwd())
init_pass_path = os.path.join(dir.parent, 'CoursewareHub/scripts/init_mdx_passwd.exp')

!chmod 775 {init_pass_path}
!{init_pass_path} {ssh_user_name} {public_ip_address} {ssh_private_key_path} {mdx_user_password}

### 既存サーバ(SSH) モード セットアップスクリプトをmdx VMへのコピー

In [ ]:
# notebookからmdx VMへのsshコマンドオプション
ssh_opts = f" -i {ssh_private_key_path} -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null"

In [ ]:
init_node_path = os.path.join(dir.parent, 'CoursewareHub/scripts/init_mdx_node.sh')

!scp {ssh_opts} {init_node_path} {ssh_user_name}@{public_ip_address}:

### 既存サーバ(SSH) モード セットアップ実行

- docker-ce インストール
- sshd_config 設定変更: Port 20022 

In [ ]:
!ssh {ssh_opts} {ssh_user_name}@{public_ip_address} chmod 775 init_mdx_node.sh
!ssh {ssh_opts} {ssh_user_name}@{public_ip_address} ./init_mdx_node.sh

### Docker のインストール確認

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{public_ip_address} sudo docker version

## sapporo-service の構築

### トークン認証のための設定
- cf. https://github.com/RCOSDP/CS-sapporo-service/blob/main/README.md#authentication

In [ ]:
import json

jwt_secret_key = 'secret123'
username = 'test_user'
password = 'test_password'

auth_config = {
    'auth_enabled': True,
    'jwt_secret_key': jwt_secret_key,
    'users': [
        {
            'username': username,
            'password': password,
        }
    ]
}

with open('auth_config.json', 'w') as f:
    json.dump(auth_config, f)

`auth_config.json` ファイルをsapporo-service実行環境にアップロードします。

In [ ]:
!scp {ssh_opts} -P 20022 auth_config.json {ssh_user_name}@{public_ip_address}:

### Docker Compose file の準備
- github.com/RCOSDP/CS-sapporo-service リポジトリから compose file を取得
- トークン認証を有効化するために compose file を修正
- sapporo-service実行環境に compose file をアップロード

In [ ]:
!curl -fsSL -O https://raw.githubusercontent.com/RCOSDP/CS-sapporo-service/main/compose.dev-deploy.yml

compose file を書き換えます。

In [ ]:
!sed -e 's|command:.*|command: sapporo --auth-config /config/auth_config.json|' \
    -e 's/127.0.0.1/0.0.0.0/' \
    -e '/volumes:/a \ \ \ \ \ \ - \$\{PWD\}\/auth_config.json:\/config\/auth_config.json' \
    -i.bak compose.dev-deploy.yml

compose fileを実行環境にアップロードします。

In [ ]:
!scp {ssh_opts} -P 20022 compose.dev-deploy.yml {ssh_user_name}@{public_ip_address}:

compose file の内容を確認します。

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{public_ip_address} cat compose.dev-deploy.yml

### sapporo-service の起動

sapporo-serviceを起動します。

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{public_ip_address} sudo -E docker compose -f compose.dev-deploy.yml up -d

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{public_ip_address} sudo -E docker compose -f compose.dev-deploy.yml logs

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{public_ip_address} curl -s localhost:1122/service-info

### 認証トークンの取得
- cf. https://github.com/RCOSDP/CS-sapporo-service/blob/main/README.md#authentication

In [ ]:
data = f'{{"username": "{username}", "password": "{password}"}}'
!curl -X POST -H "Content-Type: application/json" -d '{data}' {public_ip_address}:1122/auth

### サービス情報取得APIによるアクセス確認
取得した認証トークンを使用して接続できることを確認します。

In [ ]:
import requests

token = ''

url = f'http://{public_ip_address}:1122/service-info'
headers = {'Authorization': f'Bearer {token}'}
response = requests.get(url, headers = headers)
print(response.text)

## mdx VMの削除

ここで作成したmdx仮想マシンを削除します。

#### 仮想マシンの状態取得

In [ ]:
# 仮想マシンの状態取得
vm_info = mdx.get_vm_info(vm_name)
print(json.dumps(vm_info, indent=2, ensure_ascii=False))

#### 仮想マシンのシャットダウン、削除

In [ ]:
# 仮想マシンのシャットダウン
mdx.power_shutdown_vm(vm_name, wait_for=True)

In [ ]:
# 仮想マシンの削除
mdx.destroy_vm(vm_name, wait_for=True)

#### DNATの削除

In [ ]:
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'JWT {mdx_token}',
}

# DNAT IDの取得
dnat_id = None
url = f'https://oprpl.mdx.jp/api/dnat/project/{mdx._project_id}/'
while url and not dnat_id:
    res_dnat = requests.get(url, data=[], headers=headers)
    url = res_dnat.json()['next']
    for dnat in res_dnat.json()['results']:
        if dnat['pool_address'] == public_ip_address and dnat['dst_address'] == private_ip_address:
            dnat_id = dnat['uuid']
            break

if dnat_id:
    # DNATの削除
    url = f'https://oprpl.mdx.jp/api/dnat/{dnat_id}'
    res_dnat = requests.delete(url, headers=headers)